In [1]:
from os.path import join as pjoin
import numpy as np
import pandas as pd
from datetime import datetime 
from pytz import timezone
import pytz

In [2]:
input_filepath = "../data/raw"
train_df = pd.read_csv(input_filepath + "/train.csv")
test_df = pd.read_csv(input_filepath + "/test.csv")
building_df = pd.read_csv(input_filepath + "/building_metadata.csv")
train_df = train_df.merge(building_df, on = "building_id", how = "left")
test_df = test_df.merge(building_df, on = "building_id", how = "left")
site_info = pd.read_csv('../data/site_info.csv', delimiter = ";")

In [28]:
train_df.head()
train_df.tail()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count
20216095,1444,0,2016-12-31 23:00:00,8.750,15,Entertainment/public assembly,19619,1914.0,NaN
20216096,1445,0,2016-12-31 23:00:00,4.825,15,Education,4298,NaN,NaN
20216097,1446,0,2016-12-31 23:00:00,0.000,15,Entertainment/public assembly,11265,1997.0,NaN
20216098,1447,0,2016-12-31 23:00:00,159.575,15,Lodging/residential,29775,2001.0,NaN
20216099,1448,0,2016-12-31 23:00:00,2.850,15,Office,92271,2001.0,NaN


In [30]:
test_df.head()
test_df.tail()

,row_id,building_id,meter,timestamp,site_id,primary_use,square_feet,year_built,floor_count
41697595,41697595,1444,0,2018-05-09 07:00:00,15,Entertainment/public assembly,19619,1914.0,NaN
41697596,41697596,1445,0,2018-05-09 07:00:00,15,Education,4298,NaN,NaN
41697597,41697597,1446,0,2018-05-09 07:00:00,15,Entertainment/public assembly,11265,1997.0,NaN
41697598,41697598,1447,0,2018-05-09 07:00:00,15,Lodging/residential,29775,2001.0,NaN
41697599,41697599,1448,0,2018-05-09 07:00:00,15,Office,92271,2001.0,NaN


In [5]:
df = pd.concat([train_df, test_df])

c:\users\mufasa\desktop\uni\master biostatistik\git\ashrae-energy-prediction\env\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [9]:
train_df.shape

(20216100, 9)

In [10]:
test_df.shape

(41697600, 9)

In [11]:
df.shape

(61913700, 10)

In [12]:
train_df.shape[0] + test_df.shape[0] == df.shape[0]

True

In [26]:
df.tail()

,building_id,floor_count,meter,meter_reading,primary_use,row_id,site_id,square_feet,timestamp,year_built,timezone,country_code,location
61913695,1444,NaN,0,NaN,Entertainment/public assembly,41697595.0,15,19619,2018-05-09 07:00:00,1914.0,US/Eastern,US,"Pittsburgh, PA"
61913696,1445,NaN,0,NaN,Education,41697596.0,15,4298,2018-05-09 07:00:00,NaN,US/Eastern,US,"Pittsburgh, PA"
61913697,1446,NaN,0,NaN,Entertainment/public assembly,41697597.0,15,11265,2018-05-09 07:00:00,1997.0,US/Eastern,US,"Pittsburgh, PA"
61913698,1447,NaN,0,NaN,Lodging/residential,41697598.0,15,29775,2018-05-09 07:00:00,2001.0,US/Eastern,US,"Pittsburgh, PA"
61913699,1448,NaN,0,NaN,Office,41697599.0,15,92271,2018-05-09 07:00:00,2001.0,US/Eastern,US,"Pittsburgh, PA"


In [15]:
site_info

,site_id,timezone,country_code,location
0,0,US/Eastern,US,"Orlando, FL"
1,1,Europe/London,UK,"UK, Southhampton"
2,2,US/Mountain,US,"Tempe, AZ"
3,3,US/Eastern,US,"Washington, WA"
4,4,US/Pacific,US,"San Francisco, CA"
5,5,Europe/London,UK,"UK, London"
6,6,US/Eastern,US,Philadelphia
7,7,Canada/Eastern,CA,Montreal/Ottawa
8,8,US/Eastern,US,"Orlando, FL"
9,9,US/Central,US,"Austin, TX"


In [16]:
# convert the string column into a timezone object
site_info.timezone = site_info.timezone.map(lambda x: pytz.timezone(x))

In [17]:
site_info.timezone[1]

<DstTzInfo 'Europe/London' LMT-1 day, 23:59:00 STD>

In [18]:
train_df.shape

(20216100, 9)

In [19]:
df = df.merge(site_info, on = "site_id", how = "left")

In [34]:
del site_info

NameError: name 'site_info' is not defined

In [20]:
df.shape

(61913700, 13)

In [21]:
def reduce_mem_usage(df, verbose=True):
    """
    Takes an dataframe as argument and adjusts the datatypes of the respective
    columns to reduce memory allocation
    """
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if (c_min > np.iinfo(np.int8).min and
                        c_max < np.iinfo(np.int8).max):
                    df[col] = df[col].astype(np.int8)
                elif (c_min > np.iinfo(np.int16).min and
                      c_max < np.iinfo(np.int16).max):
                    df[col] = df[col].astype(np.int16)
                elif (c_min > np.iinfo(np.int32).min and
                      c_max < np.iinfo(np.int32).max):
                    df[col] = df[col].astype(np.int32)
                elif (c_min > np.iinfo(np.int64).min and
                      c_max < np.iinfo(np.int64).max):
                    df[col] = df[col].astype(np.int64)
            else:
                if (c_min > np.finfo(np.float16).min and
                        c_max < np.finfo(np.float16).max):
                    df[col] = df[col].astype(np.float16)
                elif (c_min > np.finfo(np.float32).min and
                      c_max < np.finfo(np.float32).max):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    reduced_mem = 100 * (start_mem - end_mem) / start_mem
    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'
              .format(end_mem, reduced_mem))
    return df


In [45]:
df = reduce_mem_usage(df)

Mem. usage decreased to 1233.89 Mb (0.0% reduction)


In [46]:
def adjust_column_types(data_frame):
    """
    Takes a data frame and parses certain columns to the desired type.
    """
    data_frame["timestamp"] = pd.to_datetime(data_frame["timestamp"])
    return data_frame

In [47]:
df = adjust_column_types(df)

In [48]:
df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,timezone,country_code,location
0,0,0,2016-01-01,0.0,0,Education,7432,2008.0,NaN,US/Eastern,US,"Orlando, FL"
1,1,0,2016-01-01,0.0,0,Education,2720,2004.0,NaN,US/Eastern,US,"Orlando, FL"
2,2,0,2016-01-01,0.0,0,Education,5376,1991.0,NaN,US/Eastern,US,"Orlando, FL"
3,3,0,2016-01-01,0.0,0,Education,23685,2002.0,NaN,US/Eastern,US,"Orlando, FL"
4,4,0,2016-01-01,0.0,0,Education,116607,1975.0,NaN,US/Eastern,US,"Orlando, FL"


In [14]:
type(df)

pandas.core.frame.DataFrame

In [49]:
#short example
ts = df['timestamp'][1]
ts

Timestamp('2016-01-01 00:00:00')

In [50]:
tz = df['timezone'][1]
tz

<DstTzInfo 'US/Eastern' LMT-1 day, 19:04:00 STD>

In [51]:
# use the timezone's localize object to turn a naive timestamp into a local one
tz.localize(ts)

Timestamp('2016-01-01 00:00:00-0500', tz='US/Eastern')

In [52]:
# minimal working example, subset of 100
mwe = df.iloc[0:99,]

In [53]:
type(mwe)

pandas.core.frame.DataFrame

In [54]:
mwe_tz = mwe['timezone']

In [55]:
mwe_tz[1]

<DstTzInfo 'US/Eastern' LMT-1 day, 19:04:00 STD>

In [56]:
mwe_ts = mwe['timestamp']

In [57]:
# for each combination of timezone and timestamp localize the respective
# timestamp 
ts_local = [tz.localize(ts) for tz, ts in zip(mwe_tz, mwe_ts)]

In [58]:
ts_local[1]

Timestamp('2016-01-01 00:00:00-0500', tz='US/Eastern')

In [59]:
mwe.loc[:,"ts_local"] = ts_local

c:\users\mufasa\desktop\uni\master biostatistik\git\ashrae-energy-prediction\env\lib\site-packages\pandas\core\indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
c:\users\mufasa\desktop\uni\master biostatistik\git\ashrae-energy-prediction\env\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [60]:
mwe.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,timezone,country_code,location,ts_local
0,0,0,2016-01-01,0.0,0,Education,7432,2008.0,NaN,US/Eastern,US,"Orlando, FL",2016-01-01 00:00:00-05:00
1,1,0,2016-01-01,0.0,0,Education,2720,2004.0,NaN,US/Eastern,US,"Orlando, FL",2016-01-01 00:00:00-05:00
2,2,0,2016-01-01,0.0,0,Education,5376,1991.0,NaN,US/Eastern,US,"Orlando, FL",2016-01-01 00:00:00-05:00
3,3,0,2016-01-01,0.0,0,Education,23685,2002.0,NaN,US/Eastern,US,"Orlando, FL",2016-01-01 00:00:00-05:00
4,4,0,2016-01-01,0.0,0,Education,116607,1975.0,NaN,US/Eastern,US,"Orlando, FL",2016-01-01 00:00:00-05:00


In [61]:
del mwe
# now for all the data #ran til here
ts = df['timestamp']

In [62]:
tz = df['timezone']

In [68]:
sub = df[['timestamp', 'timezone']]

In [69]:
sub.head

<bound method NDFrame.head of                    timestamp    timezone
0        2016-01-01 00:00:00  US/Eastern
1        2016-01-01 00:00:00  US/Eastern
2        2016-01-01 00:00:00  US/Eastern
3        2016-01-01 00:00:00  US/Eastern
4        2016-01-01 00:00:00  US/Eastern
...                      ...         ...
20216095 2016-12-31 23:00:00  US/Eastern
20216096 2016-12-31 23:00:00  US/Eastern
20216097 2016-12-31 23:00:00  US/Eastern
20216098 2016-12-31 23:00:00  US/Eastern
20216099 2016-12-31 23:00:00  US/Eastern

[20216100 rows x 2 columns]>

In [71]:
#this takes aaages 
ts_local = [a.localize(b) for a, b in zip(tz, ts)]

In [73]:
ts_local[0:10]

[Timestamp('2016-01-01 00:00:00-0500', tz='US/Eastern'),
 Timestamp('2016-01-01 00:00:00-0500', tz='US/Eastern'),
 Timestamp('2016-01-01 00:00:00-0500', tz='US/Eastern'),
 Timestamp('2016-01-01 00:00:00-0500', tz='US/Eastern'),
 Timestamp('2016-01-01 00:00:00-0500', tz='US/Eastern'),
 Timestamp('2016-01-01 00:00:00-0500', tz='US/Eastern'),
 Timestamp('2016-01-01 00:00:00-0500', tz='US/Eastern'),
 Timestamp('2016-01-01 00:00:00-0500', tz='US/Eastern'),
 Timestamp('2016-01-01 00:00:00-0500', tz='US/Eastern'),
 Timestamp('2016-01-01 00:00:00-0500', tz='US/Eastern')]

In [75]:
df.loc[:,"timestamp_local"] = ts_local

In [76]:
df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,timezone,country_code,location,timestamp_local
0,0,0,2016-01-01,0.0,0,Education,7432,2008.0,NaN,US/Eastern,US,"Orlando, FL",2016-01-01 00:00:00-05:00
1,1,0,2016-01-01,0.0,0,Education,2720,2004.0,NaN,US/Eastern,US,"Orlando, FL",2016-01-01 00:00:00-05:00
2,2,0,2016-01-01,0.0,0,Education,5376,1991.0,NaN,US/Eastern,US,"Orlando, FL",2016-01-01 00:00:00-05:00
3,3,0,2016-01-01,0.0,0,Education,23685,2002.0,NaN,US/Eastern,US,"Orlando, FL",2016-01-01 00:00:00-05:00
4,4,0,2016-01-01,0.0,0,Education,116607,1975.0,NaN,US/Eastern,US,"Orlando, FL",2016-01-01 00:00:00-05:00


In [79]:
#df.to_pickle("../data/test_train_converted.pkl")

In [22]:
#meter = pd.read_pickle("../data/test_train_converted.pkl")

In [25]:
#del meter

In [82]:
weather = pd.read_pickle("../data/weather_converted.pkl")

In [ ]:
weather.rename(columns={'timestamp': 'timestamp_weather_utc'}, inplace = True)

In [85]:
weather.head()

,site_id,timestamp_weather_utc,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,timezone,country_code,location,timestamp_local
0,0,2016-01-01 00:00:00+00:00,25.000000,6.0,20.0,NaN,1019.700012,0.0,0.0,US/Eastern,US,"Orlando, FL",2015-12-31 19:00:00-05:00
1,0,2016-01-01 01:00:00+00:00,24.400000,NaN,21.1,-1.0,1020.200012,70.0,1.5,US/Eastern,US,"Orlando, FL",2015-12-31 20:00:00-05:00
2,0,2016-01-01 02:00:00+00:00,22.799999,2.0,21.1,0.0,1020.200012,0.0,0.0,US/Eastern,US,"Orlando, FL",2015-12-31 21:00:00-05:00
3,0,2016-01-01 03:00:00+00:00,21.100000,2.0,20.6,0.0,1020.099976,0.0,0.0,US/Eastern,US,"Orlando, FL",2015-12-31 22:00:00-05:00
4,0,2016-01-01 04:00:00+00:00,20.000000,2.0,20.0,-1.0,1020.000000,250.0,2.6,US/Eastern,US,"Orlando, FL",2015-12-31 23:00:00-05:00


In [99]:
all_data = df.merge(weather, how = "outer", on = ["site_id","timezone","country_code", "location", "timestamp_local"])

In [102]:
weather.shape

(417016, 13)

In [103]:
df.shape

(20216100, 13)

In [101]:
all_data.head().iloc[:,1:15]

,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,timezone,country_code,location,timestamp_local,timestamp_weather_utc,air_temperature
0,0.0,2016-01-01,0.0,0,Education,7432.0,2008.0,NaN,US/Eastern,US,"Orlando, FL",2016-01-01 00:00:00-05:00,2016-01-01 05:00:00+00:00,19.4
1,0.0,2016-01-01,0.0,0,Education,2720.0,2004.0,NaN,US/Eastern,US,"Orlando, FL",2016-01-01 00:00:00-05:00,2016-01-01 05:00:00+00:00,19.4
2,0.0,2016-01-01,0.0,0,Education,5376.0,1991.0,NaN,US/Eastern,US,"Orlando, FL",2016-01-01 00:00:00-05:00,2016-01-01 05:00:00+00:00,19.4
3,0.0,2016-01-01,0.0,0,Education,23685.0,2002.0,NaN,US/Eastern,US,"Orlando, FL",2016-01-01 00:00:00-05:00,2016-01-01 05:00:00+00:00,19.4
4,0.0,2016-01-01,0.0,0,Education,116607.0,1975.0,NaN,US/Eastern,US,"Orlando, FL",2016-01-01 00:00:00-05:00,2016-01-01 05:00:00+00:00,19.4


In [98]:
all_data.tail().iloc[:,1:13]

,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,timezone,country_code,location,timestamp_local
20216095,0,2016-12-31 23:00:00,8.750000,15,Entertainment/public assembly,19619,1914.0,NaN,US/Eastern,US,"Pittsburgh, PA",2016-12-31 23:00:00-05:00
20216096,0,2016-12-31 23:00:00,4.825000,15,Education,4298,NaN,NaN,US/Eastern,US,"Pittsburgh, PA",2016-12-31 23:00:00-05:00
20216097,0,2016-12-31 23:00:00,0.000000,15,Entertainment/public assembly,11265,1997.0,NaN,US/Eastern,US,"Pittsburgh, PA",2016-12-31 23:00:00-05:00
20216098,0,2016-12-31 23:00:00,159.574997,15,Lodging/residential,29775,2001.0,NaN,US/Eastern,US,"Pittsburgh, PA",2016-12-31 23:00:00-05:00
20216099,0,2016-12-31 23:00:00,2.850000,15,Office,92271,2001.0,NaN,US/Eastern,US,"Pittsburgh, PA",2016-12-31 23:00:00-05:00
